In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels import stats as sm_stats
from statsmodels.stats.multitest import fdrcorrection
import pingouin as pg

import seaborn as sns
import pylab as plt
from statannotations.Annotator import Annotator

from importlib import reload
from functools import reduce
import datetime
import re

import sys
sys.path.insert(1,'../scripts')
import utils
import plots

In [ ]:
data_path = '/rds/general/user/aschalka/home/data/ppmi'
path = '/rds/general/user/aschalka/home/data/ppmi/accelerometer'
img_path = '/rds/general/user/aschalka/home/images/paper/prodromalPPMI'

In [ ]:
merged = pd.read_csv(f'{path}/residuals.csv',index_col=0) # created by scripts/run_feature_extraction.sh
converters = pd.read_csv('/rds/general/user/aschalka/home/data/ppmi/analyses/prodromal/converterInfo.csv')
converters = converters[converters['phenoconverted']==1].groupby('participant').first().index

In [ ]:
predictors_res = merged.filter(regex='_residual').columns
labels = pd.Series(np.hstack([np.repeat('physical activity',2),np.repeat('sleep',8),np.repeat('vital',4)]),index=predictors_res)
color_map = dict(zip(np.hstack([np.arange(8),np.unique(labels)]),sns.color_palette('deep')))
color_map2 = dict(zip(np.unique(labels),sns.color_palette('bright')))

In [ ]:
clean = merged[merged['diagnosis'].isin(['pd','hc','prod'])]
# remove converters
clean = clean.drop(index=converters)

In [ ]:
clean['male sex'] = clean['gender'].replace(['f','m'],[0,1])
clean = clean.dropna(subset=predictors_res,how='all')
print(clean.groupby('diagnosis')[['male sex','age_accelerometry_mean']].agg(['mean','std','size']).dropna().loc[['hc','pd','prod']].to_latex())

In [ ]:
clean.groupby(['diagnosis','male sex'])[['age_accelerometry_mean']].agg(['mean','std','size'])

In [ ]:
pg.ttest(clean.loc[clean['diagnosis']=='prod','age_accelerometry_mean'],clean.loc[clean['diagnosis']=='pd','age_accelerometry_mean'])

In [ ]:

clean = merged[merged['diagnosis'].isin(['pd','hc','prod'])]
# remove converters
clean = clean.drop(index=converters)
ttests = pd.DataFrame(columns=pd.MultiIndex.from_product([predictors_res,['t','p','N1','N2']],names=['feature','statistic']),
                      index=pd.MultiIndex.from_product([['hc','prod','pd'],['hc','prod','pd']],
                                                              names = ['g1','g2']))

fig, axes = plt.subplots(nrows=len(predictors_res)//5+1,ncols=5,figsize=(20,10))
plots.plot_context()

for ax,pred in zip(axes.flatten(),predictors_res):
    predname = re.sub(r'_(\w)', r' \1', pred) # replace underscores with spaces
    predname = re.sub(r' residual$', r'\nresidual', predname)
    print(predname)
    sns.boxplot(y=pred,x='diagnosis',data=clean,ax=ax,order=['pd','prod','hc'])
    thresh = 0.05/3
    annot = [[2e-6, "****"], [2e-4, "***"], [2e-3, "**"], [thresh, "*"], [1, "ns"]]
    box_pairs = []
    ttest = pd.DataFrame(index=pd.MultiIndex.from_product([['hc','prod','pd'],['hc','prod','pd']],
                                                              names = ['g1','g2']),columns=['t','p'])
    for i,g1 in enumerate(['hc','prod','pd']):
        for j,g2 in enumerate(['hc','prod','pd']):
            if j>i:
                t,p = stats.ttest_ind(clean[clean['diagnosis']==g1][pred].dropna(),
                                      clean[clean['diagnosis']==g2][pred].dropna())
                ttest.loc[(g1,g2),'t'] = t
                ttest.loc[(g1,g2),'p'] = p
                ttests.loc[(g1,g2),(pred,'t')] = t
                ttests.loc[(g1,g2),(pred,'p')] = p
                ttests.loc[(g1,g2),(pred,'N1')] = clean[clean['diagnosis']==g1][pred].dropna().shape[0]
                ttests.loc[(g1,g2),(pred,'N2')] = clean[clean['diagnosis']==g2][pred].dropna().shape[0]
    ttest = ttest.dropna(axis='rows',how='all')
    sign = ttest[ttest['p']<thresh]
    for key,row in sign.iterrows():
        box_pairs.append((key[0],key[1]))
    if len(box_pairs)>0:
        ax, test_results = add_stat_annotation(ax, data=clean, x='diagnosis', y=pred,order=['pd','prod','hc'],
                                       box_pairs=box_pairs,
                                       test='t-test_ind', text_format='star', loc='outside', verbose=2,comparisons_correction=None,
                                          pvalue_thresholds=annot)
    ax.set_ylabel(predname,color=color_map[labels[pred]])
    N_diagnosis = clean.dropna(subset=[pred]).groupby(['diagnosis']).size()
    N_diagnosis = N_diagnosis.loc[['pd','prod','hc']]
    plots.add_median_labels(ax,N_diagnosis,fmt="%d",remove=len(box_pairs))

#axes[-1, -1].remove()
#legend_ax = fig.add_subplot(224)

# Plot dummy data in the legend subplot to create the custom legend
cmap = {k: v for k, v in color_map.items() if k in ['physical activity', 'sleep', 'vital']}
for label, color in cmap.items():
    axes[-1,-1].plot([], [], color=color, label=f'{label}', linewidth=10)

# Hide the legend subplot frame and ticks
axes[-1,-1].set_frame_on(False)
axes[-1,-1].tick_params(left=False, right=False, top=False, bottom=False)
axes[-1,-1].set_xticks([])
axes[-1,-1].set_yticks([])

# Create the legend in the bottom right corner
axes[-1,-1].legend(loc='lower right', frameon=False)
        
plt.tight_layout()
#plt.savefig('/scratch/c.c21013066/images/ppmi/studywatch/mean_digital_boxplot_diag_residual.png',dpi=300,bbox_inches='tight')
#plt.savefig('/scratch/c.c21013066/images/ppmi/studywatch/mean_digital_boxplot_diag_residual.pdf',dpi=300,bbox_inches='tight')

In [ ]:
ttests.to_csv('/scratch/c.c21013066/data/ppmi/analyses/studywatch/digitalmeanresid_groupdiff.csv')

In [ ]:
prodromal = pd.read_csv(f'{data_path}/analyses/prodromal/Heinzel_Yan2024_>1.csv').set_index('participant')

digprod = pd.merge(prodromal,merged,right_index=True,left_index=True,how='outer',suffixes=['','_drop'])
digprod = digprod.drop(columns=digprod.filter(regex='_drop').columns).reset_index().rename(columns={'index':'participant'})

saa_positive = pd.read_csv(f'{data_path}/phenotypes2021/biospecimen_SAA_all_clean.csv')
saa_positive['SAA_positive'] = saa_positive['SAA_positive'].replace([0,1],['no','yes'])
saa_positive = pd.concat([saa_positive,pd.get_dummies(saa_positive['SAA_positive'],prefix='SAA')],axis=1)
digprod = pd.merge(digprod,saa_positive,on='participant',how='outer')

digprod['positiveDaT_yes'] = digprod['dat_deficit_visual']

traits = ['GBA','SNCA','LRRK2','HYPOSMIA_yes','rbd_psgproven_yes','positiveDaT_yes','SAA_yes']
trait_names = ['GBA','SNCA','LRRK2','hyposmia','RBD','positive\nDaTscan','positive\nSAA']
digprod = digprod.set_index('participant')
df = digprod[digprod['diagnosis'].isin(['pd','hc'])]
dat = digprod.loc[converters]
dat['diagnosis'] = 'converted'
df = pd.concat([df,dat[dat['diagnosis']=='converted']])
dat = digprod.loc[digprod['diagnosis']=='prod']
dat = dat.drop(index=converters)
df = pd.concat([df,dat[dat['diagnosis']=='prod']])
dat['diagnosis'] = 'converted'
df = pd.concat([df,dat[dat['diagnosis']=='converted']])
for name,trait in zip(trait_names,traits):
    dat = digprod[(digprod['diagnosis'] == 'prod') & (digprod[trait] == 1.0)]

    # Then modify the 'diagnosis' column based on those temporary columns
    dat['diagnosis'] = name
    df = pd.concat([df,dat[dat['diagnosis']==name]])

In [ ]:
df['male sex'] = df['gender'].replace(['f','m'],[0,1])
#df = df.dropna(subset=predictors_res,how='all')
print(df.dropna(subset='age_accelerometry_mean').groupby('diagnosis')[['male sex','age_accelerometry_mean']].agg(['mean','std','size']).dropna().loc[['hc','pd','prod','LRRK2','GBA','hyposmia','RBD','positive\nDaTscan',
                                                                                                             'positive\nSAA']].to_latex())

In [ ]:
table = df.dropna(subset='age_accelerometry_mean').groupby('diagnosis')[['male sex','age_accelerometry_mean']].agg(['mean','std','size']).dropna().loc[['hc','pd','prod','LRRK2','GBA','hyposmia','RBD','positive\nDaTscan',
                                                                                                             'positive\nSAA']]

In [ ]:
df.dropna(subset='age_accelerometry_mean').groupby('diagnosis')[['male sex','age_accelerometry_mean']].agg(['mean','std','count']).dropna().loc[['hc','pd','prod','LRRK2','GBA','hyposmia','RBD','positive\nDaTscan',
                                                                                                             'positive\nSAA']].to_csv(f'{data_path}/analyses/prodromal/demographics.csv')

In [ ]:
nona = df.dropna(subset='age_accelerometry_mean')
pg.ttest(nona.loc[nona['diagnosis']=='prod','age_accelerometry_mean'],nona.loc[nona['diagnosis']=='pd','age_accelerometry_mean'])

In [ ]:
fig, axes = plt.subplots(nrows=len(predictors_res)//4+1,ncols=4,figsize=(18,20))
plots.plot_context()
ttests = pd.DataFrame(index=pd.MultiIndex.from_product([['hc','pd','prod','GBA','LRRK2','hyposmia','RBD','positive\nDaTscan',
                                                        'positive\nSAA'],['pd','hc','prod',
                                                                                                                                         'GBA','LRRK2','hyposmia',
                                                                                                                                      'RBD','positive\nDaTscan','positive\nSAA']],
                                                              names = ['g1','g2']),columns=pd.MultiIndex.from_product([predictors_res,['T','dof','alternative','p-val','CI95%','Cohen-d','BF10','power',
                                                                                                                                      'FDR corrected p-val']],names=['marker','statistic']))

for ax,pred in zip(axes.flatten(),predictors_res):
    predname = re.sub(r'_(\w)', r' \1', pred)
    predname = re.sub(r' residual$', r'\nresidual', predname)
    print(predname)
    sns.boxplot(y=pred,x='diagnosis',data=df,ax=ax,order=['pd','hc','prod','GBA','LRRK2','hyposmia','RBD','positive\nDaTscan','positive\nSAA'])
    thresh = 0.05/(2*len(['prod','GBA','LRRK2','hyposmia','RBD','positive\nDaTscan',
                                                        'positive\nSAA']))
    print(thresh)
    sns.stripplot(y=pred,x='diagnosis',data=df,ax=ax,order=['pd','hc','prod','GBA','LRRK2','hyposmia','RBD','positive\nDaTscan','positive\nSAA'])
    annot = [[3.6e-6, "****"], [3.6e-5, "***"], [3.6e-4, "**"], [thresh, "*"], [1, "ns"]]
    box_pairs = []
    ttest = pd.DataFrame(index=pd.MultiIndex.from_product([['hc','pd','prod','GBA','LRRK2','hyposmia','RBD','positive\nDaTscan','positive\nSAA'],['pd','hc','prod',
                                                                                                                                         'GBA','LRRK2','hyposmia','RBD',
                                                                                                                                         'positive\nDaTscan','positive\nSAA']],
                                                              names = ['g1','g2']),columns=['T','dof','alternative','p-val','CI95%','Cohen-d','BF10','power','FDR corrected p-val'])
    for i,g1 in enumerate(['pd','hc']):#,'GBA','LRRK2','olfactory\nloss','RBD','positive\nDaTscan','positive\nSAA']):
        for j,g2 in enumerate(['pd','hc','prod','GBA','LRRK2','hyposmia','RBD','positive\nDaTscan','positive\nSAA']):
            if j>i:
                t,p = stats.ttest_ind(df[df['diagnosis']==g1][pred].dropna(),
                                      df[df['diagnosis']==g2][pred].dropna())
                ttest.loc[(g1,g2),['T','dof','alternative','p-val','CI95%','Cohen-d','BF10','power']] =                 pg.ttest(df[df['diagnosis']==g1][pred].dropna(),
                                      df[df['diagnosis']==g2][pred].dropna(),correction=False).values
    ttest['FDR corrected p-val'] = fdrcorrection(ttest['p-val'], alpha=0.05, method='indep', is_sorted=False)[1]
    ttests.loc[:,(pred,slice(None))] = ttest.values
    sign = ttest[ttest['FDR corrected p-val']<0.05]
    pvals = sign['FDR corrected p-val']
    formatted_pvals = ['{:.2e}'.format(num) for num in pvals]
    for key,row in sign.iterrows():
        box_pairs.append((key[0],key[1]))
    if len(box_pairs)>0:

        annotator = Annotator(ax, box_pairs, data=df, x='diagnosis', y=pred,order=['pd','hc','prod','GBA','LRRK2','hyposmia','RBD','positive\nDaTscan','positive\nSAA'],
                                      perform_stat_test=False)
        annotator.configure(test=None, text_format='full', loc='inside',text_offset=0.5,
                            verbose=2,comparisons_correction=None,line_offset=1.5,fontsize=12).set_pvalues(pvalues=pvals).set_custom_annotations(formatted_pvals)
        annotator.annotate()
    N_diagnosis = df.dropna(subset=[pred]).groupby(['diagnosis']).size()
    N_diagnosis = N_diagnosis.loc[['pd','hc','prod','GBA','LRRK2','hyposmia','RBD','positive\nDaTscan','positive\nSAA']]
    plots.add_median_labels(ax,N_diagnosis,fmt="%d",remove=len(box_pairs))
    ax.set_ylabel(predname,color=color_map2[labels[pred]])
    ax.set_xlabel('')
    ax.set_xticklabels(['PD','HC','at-risk','GBA','LRRK2','hyposmia','RBD','DaT+','SAA+'],rotation=90)
    if pred==predictors_res[-1]:
        pass
    else:
        ax.legend([],[],frameon=False)
    ttests.loc[:,(pred,slice(None))] = ttest.values
    ax.axvline(2.5,linestyle='--',color='k')

cmap = {k: v for k, v in color_map2.items() if k in ['physical activity', 'sleep', 'vital']}
for label, color in cmap.items():
    axes[-1,-1].plot([], [], color=color, label=f'{label}', linewidth=10)

axes[-1,-1].set_frame_on(False)
axes[-1,-1].tick_params(left=False, right=False, top=False, bottom=False)
axes[-1,-1].set_xticks([])
axes[-1,-1].set_yticks([])
axes[-1,-2].set_frame_on(False)
axes[-1,-2].tick_params(left=False, right=False, top=False, bottom=False)
axes[-1,-2].set_xticks([])
axes[-1,-2].set_yticks([])

axes[-1,-1].legend(loc='lower right', frameon=False)

plt.tight_layout()
plt.savefig(f'{img_path}/mean_digital_boxplot_diag_residual_prod_seqdata_Yan2024.png',dpi=300,bbox_inches='tight')
plt.savefig(f'{img_path}/mean_digital_boxplot_diag_residual_prod_seqdata_yan2024.pdf',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
ttests.dropna().loc[('pd','hc'),(slice(None),['Cohen-d','FDR corrected p-val'])]

In [ ]:
ttests['sleep_efficiency_residual'][['Cohen-d','FDR corrected p-val']].dropna()

In [ ]:
fig = plt.figure(figsize=(10,5))
plots.plot_context()

for pred in ['sleep_efficiency_residual']:
    ttest = ttests[pred]
    predname = re.sub(r'_(\w)', r' \1', pred) # replace underscores with spaces
    predname = re.sub(r' residual$', r'\nresidual', predname)
    print(predname)
    ax=sns.boxplot(y=pred,x='diagnosis',data=df,order=['pd','hc','prod','GBA','LRRK2','hyposmia','RBD','positive\nDaTscan','positive\nSAA'])
    sign = ttest[ttest['FDR corrected p-val']<0.05]
    pvals = sign['FDR corrected p-val']
    formatted_pvals = ['{:.2e}'.format(num) for num in pvals]
    for key,row in sign.iterrows():
        box_pairs.append((key[0],key[1]))
    if len(box_pairs)>0:
        annotator = Annotator(ax, box_pairs, data=df, x='diagnosis', y=pred,order=['pd','hc','prod','GBA','LRRK2','hyposmia','RBD','positive\nDaTscan','positive\nSAA'],
                                      perform_stat_test=False)
        annotator.configure(test=None, text_format='full', loc='outside',
                            verbose=2,comparisons_correction=None,line_offset=0.5,fontsize=12).set_pvalues(pvalues=pvals).set_custom_annotations(formatted_pvals)
        annotator.annotate()
    N_diagnosis = df.dropna(subset=[pred]).groupby(['diagnosis']).size()
    N_diagnosis = N_diagnosis.loc[['pd','hc','prod','GBA','LRRK2','hyposmia','RBD','positive\nDaTscan','positive\nSAA']]
    plots.add_median_labels(ax,N_diagnosis,fmt="%d",remove=len(box_pairs))
    ax.set_ylabel(predname,color=color_map2[labels[pred]])
    ax.set_xlabel('')
    ax.set_xticklabels(['PD','HC','at-risk','GBA','LRRK2','hyposmia','RBD','DaT+','SAA+'],rotation=90)
    if pred==predictors_res[-1]:
        pass
    else:
        ax.legend([],[],frameon=False)
    ax.axvline(2.5,linestyle='--',color='k')

plt.savefig(f'{img_path}/mean_digital_boxplot_sleepeff.png',dpi=300,bbox_inches='tight')
plt.savefig(f'{img_path}/mean_digital_boxplot_sleepeff.pdf',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
ttests.to_csv(f'{data_path}/analyses/studywatch/digitalmeanresid_groupdiff_prod_Yan2024.csv')

In [ ]:
ttests = pd.read_csv(f'{data_path}/analyses/studywatch/digitalmeanresid_groupdiff_prod_Yan2024.csv',index_col=[0,1],header=[0,1])

In [ ]:
ttests = ttests.dropna()

In [ ]:
for feature in ttests.columns.levels[0]:
    print(sm_stats.multitest.fdrcorrection(ttests[(feature,'p-val')], alpha=0.05, method='indep', is_sorted=False)[1])
    ttests[(feature,'FRD corrected p-val')] = sm_stats.multitest.fdrcorrection(ttests[(feature,'p-val')], alpha=0.05, method='indep', is_sorted=False)[1]